# カテゴリ型

In [1]:
import sys
import numpy as np
import pandas as pd

sys.path.append("../src")
from data_loader import load_hotel_reserve, load_production, load_production_missing_category

customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()
production = load_production()
production_missc_tb = load_production_missing_category()


## カテゴリ型への変換
カテゴリ型にすることで文字列で持っているよりもデータサイズを小さくできる。  
顧客テーブルの性別をブール型とカテゴリ型に変換する。

`astype`関数に"bool", "category"を指定することでそれぞれに変換できる。

In [2]:
# sexがmanのときにTRUEとするブール型を追加
# このコードは、as.type関数を利用しなくてもブール型に変換
customer_tb[['sex_is_man']] = (customer_tb[['sex']] == 'man').astype('bool')

# sexをカテゴリ型に変換
customer_tb['sex_c'] = \
  pd.Categorical(customer_tb['sex'], categories=['man', 'woman'])

# astype関数でも変換可能
# customer_tb['sex_c'] = customer_tb['sex_c'].astype('category')

# インデックスデータはcodesに格納されている
print(customer_tb['sex_c'].cat.codes)

# マスタデータはcategoriesに格納されている
print(customer_tb['sex_c'].cat.categories)

0      0
1      0
2      1
3      0
4      0
      ..
995    0
996    0
997    1
998    1
999    0
Length: 1000, dtype: int8
Index(['man', 'woman'], dtype='object')


In [3]:
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c
0,c_1,41,man,35.092193,136.512347,True,man
1,c_2,38,man,35.325076,139.410551,True,man
2,c_3,49,woman,35.120543,136.511179,False,woman
3,c_4,43,man,43.034868,141.240314,True,man
4,c_5,31,man,35.102661,136.523797,True,man


## ダミー変数化
MLに用いられる一部のメソッドがカテゴリ型に対応していない場合があるため、カテゴリ値をフラグ集合値に変換する**ダミー変数化**が必要になる。

顧客テーブルの性別をダミー変数化する。

In [4]:
# ダミー変数化する前にカテゴリ型に変換
customer_tb['sex'] = pd.Categorical(customer_tb['sex'])

# get_dummies関数によってsexをダミー変数化
# drop_firstをFalseにすると、カテゴリ値の全種類の値のダミーフラグを生成
dummy_vars = pd.get_dummies(customer_tb['sex'], drop_first=False)

In [5]:
dummy_vars.head()

,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


## カテゴリ値の集約
データ数が極端に少ないカテゴリ値は、他のカテゴリ値とまとめることがある。  
顧客テーブルの年齢を10才区切りでカテゴリ型に変換し、さらに60才以上の場合は"60才以上"というカテゴリ値に変換する。

category型のマスタデータの更新には、`add_categories`関数と`remove_unused_categories`関数を利用する。

In [6]:
# pd.Categoricalでcategory型に変換
customer_tb["age_rank"] = \
    pd.Categorical(np.floor(customer_tb["age"] / 10) * 10)

# マスタデータに"60以上"を追加
customer_tb["age_rank"] = customer_tb["age_rank"].cat.add_categories(["60以上"])

# 集約するデータを書き換え
# category型は、=または!=判定のみ可能なので、isis関数を利用
customer_tb.loc[customer_tb["age_rank"].isin(
    [60.0, 70.0, 80.0]), "age_rank"] = "60以上"
# 利用されていないマスタデータを削除
customer_tb["age_rank"] = customer_tb["age_rank"].cat.remove_unused_categories()


In [7]:
# 1回で書く方法
customer_tb["age_rank_better"] = \
    pd.Series(np.floor(customer_tb["age"] / 10) * 10) \
    .apply(lambda x: "60以上" if x in [60.0, 70.0, 80.0] else x) \
    .astype("category")


In [8]:
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,age_rank,age_rank_better
0,c_1,41,man,35.092193,136.512347,True,man,40.0,40.0
1,c_2,38,man,35.325076,139.410551,True,man,30.0,30.0
2,c_3,49,woman,35.120543,136.511179,False,woman,40.0,40.0
3,c_4,43,man,43.034868,141.240314,True,man,40.0,40.0
4,c_5,31,man,35.102661,136.523797,True,man,30.0,30.0


## カテゴリ値の組み合わせ
顧客テーブルの性別と年齢の10才区切りのカテゴリ値を組み合わせて、性別/年代のカテゴリ値を生成する。

In [9]:
customer_tb['sex_and_age'] = pd.Categorical(
    # 連結する列を抽出
    customer_tb[['sex', 'age']]

    # lambda関数内でsexと10代区切りのageを_を挟んで文字列として連結
    .apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1] / 10) * 10),
           axis=1)
)


In [10]:
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,age_rank,age_rank_better,sex_and_age
0,c_1,41,man,35.092193,136.512347,True,man,40.0,40.0,man_40.0
1,c_2,38,man,35.325076,139.410551,True,man,30.0,30.0,man_30.0
2,c_3,49,woman,35.120543,136.511179,False,woman,40.0,40.0,woman_40.0
3,c_4,43,man,43.034868,141.240314,True,man,40.0,40.0,man_40.0
4,c_5,31,man,35.102661,136.523797,True,man,30.0,30.0,man_30.0


## カテゴリ型の数値化
カテゴリ型の集約方法の一つとして数値化する方法がある。学習データが少ない場合において、カテゴリ値を考慮して予測モデルを作りたいときに利用することが多い。ただし、データの本来の意味を失ったり過学習を引き起こすことがあるので注意が必要。

例）
- 製造物の品種ごとにレコード内の出現回数をカウントし、カテゴリ値の代わりに利用
- 製造物の品種ごとに製造障害率（障害が発生した割合）を計算し、カテゴリ値の代わりに利用
- カテゴリ値ごとの製造障害率を基準に、カテゴリ値ごとの障害発生率の高い順に順位を計算し、カテゴリ値の代わりに使用

以下では製品種別(type)を製品種別ごとの平均障害率に変換する。ただし、平均障害率の計算は自身のレコードを除いて計算する。

In [11]:
# 製品種別ごとの障害数
fault_cnt_per_type = production \
    .query('fault_flg') \
    .groupby('type')['fault_flg'] \
    .count()

# 製品種別ごとの製造数
type_cnt = production.groupby('type')['fault_flg'].count()

production['type_fault_rate'] = production[['type', 'fault_flg']] \
    .apply(lambda x:
           (fault_cnt_per_type[x[0]] - int(x[1])) / (type_cnt[x[0]] - 1),
           axis=1)


In [12]:
fault_cnt_per_type

type
A    11
B     6
C    16
D     7
E    12
Name: fault_flg, dtype: int64

In [13]:
type_cnt

type
A    202
B    175
C    211
D    215
E    197
Name: fault_flg, dtype: int64

In [14]:
production.head()

,type,length,thickness,fault_flg,type_fault_rate
0,E,274.027383,40.241131,False,0.061224
1,D,86.319269,16.906715,False,0.032710
2,E,123.940388,1.018462,False,0.061224
3,B,175.554886,16.414924,False,0.034483
4,B,244.934740,29.061081,False,0.034483


## カテゴリ型の補完
1. 固定値によって補完
2. 集計値によって補完
3. 欠損していないデータに基づく予測値によって補完
4. 時系列の関係から補完
5. 多重代入法
6. 最尤法

fault_flgに欠損が存在する製造レコードを対象とする。fault_flgが欠損していないデータを用いた予測結果から、欠損しているfault_flgを補完する。予測にはKNNを用いる。

In [15]:
from sklearn.neighbors import KNeighborsClassifier

# replace関数によって、Noneをnanに変換
production_missc_tb.replace('None', np.nan, inplace=True)

# 欠損していないデータの抽出
train = production_missc_tb.dropna(subset=['type'])

# 欠損しているデータの抽出
test = production_missc_tb \
    .loc[production_missc_tb.index.difference(train.index), :]

# knnモデル生成、n_neighborsはknnのkパラメータ
kn = KNeighborsClassifier(n_neighbors=3)

# knnモデル学習
kn.fit(train[['length', 'thickness']], train['type'])

# knnモデルによって予測値を計算し、typeを補完
test['type'] = kn.predict(test[['length', 'thickness']])


In [16]:
production_missc_tb["fault_flg"].value_counts()

False    948
True      52
Name: fault_flg, dtype: int64